# title

In [123]:
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [2]:
def get_video_urls(youtube_playlist_url):
    #webdriver setup & html scrape
    driver = webdriver.Chrome('resources/chromedriver')
    driver.get(youtube_playlist_url)
    soup = BeautifulSoup(driver.page_source)

    #isolate the video stats
    video_stats = soup.find_all('yt-formatted-string')[5].text
    #use regex to isolate video number
    no_of_videos = re.sub(',', '', video_stats)
    no_of_videos = re.sub(' videos', '', video_stats)
    #number of scroll based on 100 thumbnails per scroll
    no_of_scrolls = int(int(no_of_videos) / 100 + 1)

    #scroll page for every 100 videos
    for i in range(no_of_scrolls):
        driver.execute_script("window.scrollBy(0, 12000);")
        time.sleep(3)

    soup = BeautifulSoup(driver.page_source)
    driver.quit()

    #extract youtube playlist urls
    matches = soup.find_all('a', class_='yt-simple-endpoint style-scope ytd-playlist-video-renderer')
    youtube_urls = []
    for url in matches:
        string = url.get('href')
        substring = re.search(r'\/watch\?v=\w*', string).group()
        youtube_urls.append('http://www.youtube.com' + substring)    
    return youtube_urls

- title
- url
- account name
- subscribers
- views
- date uploaded
- number of likes
- number of dislikes

In [170]:
def get_stats(youtube_url):
    driver = webdriver.Chrome('resources/chromedriver')
    driver.get(youtube_url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source)
    driver.quit()
    
    def km_cleaner(value):
        if "subscribers" in value:
            if 'K' in value:
                value = float(re.sub('K subscribers', '', value))
                value *= 1000
                return int(value)
            elif 'M' in value:
                value = float(re.sub('M subscribers', '', value))
                value *= 1000000
                return int(value)
            else:
                return int(value)
        elif "views" in value:
            if 'K' in value:
                value = float(re.sub('K views', '', value))
                value *= 1000
                return int(value)
            elif 'M' in value:
                value = float(re.sub('M views', '', value))
                value *= 1000000
                return int(value)
            else:
                return int(value)
        else:
            if 'K' in value:
                value = float(re.sub('K', '', value))
                value *= 1000
                return int(value)
            elif 'M' in value:
                value = float(re.sub('M', '', value))
                value *= 1000000
                return int(value)
            else:
                return int(value)
            
    title_value = soup.find('h1').text
    
    account_value = soup.find('yt-formatted-string', class_='ytd-channel-name').text
    
    subs_string = soup.find('yt-formatted-string', id='owner-sub-count').text
    subs_value = km_cleaner(subs_string)
    
    view_count_string = soup.find('span', class_='short-view-count').text
    view_count = km_cleaner(view_count_string)
    
    def date_uploaded(value):
        value = re.sub('•', '', value)
        if 'ago' in value:
            value = int(re.search(r'\d+', value).group())
            uploaded = (datetime.now() - timedelta(hours=value))
            return datetime.strftime(uploaded, '%b %-d, %Y')
        else:
            return value
    
    date_string = soup.find('div', id='date').text
    date_value = date_uploaded(date_string)
    
    likes_value = soup.find_all('yt-formatted-string', class_='ytd-toggle-button-renderer')[0].text
    dislikes_value = soup.find_all('yt-formatted-string', class_='ytd-toggle-button-renderer')[1].text

    
    stats = {'account': account_value,
             'video_title': title_value,
             'date_uploaded': date_value,
             'views': view_count,
             'subscribers': subs_value,
             'likes': likes_value,
             'dislikes': dislikes_value,
             'url': youtube_url
            }
    
    return stats 

In [171]:
get_stats('https://www.youtube.com/watch?v=JmRO_0jat5U')

{'video_title': 'SUMMER NIGHTS CityPop シティポップ 80s Mix',
 'date_uploaded': 'Aug 12, 2019',
 'views': 92000,
 'account': 'Ganymede Cafe',
 'subscribers': 21200,
 'likes': '2.8K',
 'dislikes': '19',
 'url': 'https://www.youtube.com/watch?v=JmRO_0jat5U'}